In [1]:
!pip install rasterio geopandas rasterstats matplotlib shapely fiona pyproj

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 94.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [9]:
data_dir = '/content/drive/MyDrive/'

b04_path = data_dir + 'B04.tiff'
b08_path = data_dir + 'B08.tiff'
shapefile_path = data_dir + 'london.shp'

In [10]:
import rasterio
import numpy as np

with rasterio.open(b08_path) as nir_src:
    nir = nir_src.read(1).astype('float32')
    nir_profile = nir_src.profile

with rasterio.open(b04_path) as red_src:
    red = red_src.read(1).astype('float32')

ndvi = (nir - red) / (nir + red + 1e-10)
ndvi = np.clip(ndvi, -1, 1)


In [11]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

dst_crs = 'EPSG:27700'
transform, width, height = calculate_default_transform(
    nir_profile['crs'], dst_crs, nir_profile['width'], nir_profile['height'], *nir_src.bounds)

reprojected_profile = nir_profile.copy()
reprojected_profile.update({
    'crs': dst_crs,
    'transform': transform,
    'width': width,
    'height': height
})

ndvi_reprojected = np.empty((height, width), dtype='float32')

reproject(
    source=ndvi,
    destination=ndvi_reprojected,
    src_transform=nir_profile['transform'],
    src_crs=nir_profile['crs'],
    dst_transform=transform,
    dst_crs=dst_crs,
    resampling=Resampling.nearest
)


(array([[0.7024352 , 0.70182633, 0.7620241 , ..., 0.79111075, 0.66784513,
         0.47726834],
        [0.25938693, 0.34177583, 0.77499384, ..., 0.31794888, 0.62883765,
         0.7730065 ],
        [0.33508438, 0.4333162 , 0.7184322 , ..., 0.42193177, 0.5619465 ,
         0.8203759 ],
        ...,
        [0.28586283, 0.36009094, 0.72316605, ..., 0.43663612, 0.3219273 ,
         0.7486462 ],
        [0.4230029 , 0.47061443, 0.370042  , ..., 0.58910704, 0.2478294 ,
         0.65399677],
        [0.29730508, 0.3868126 , 0.5563536 , ..., 0.26113224, 0.54799986,
         0.5226558 ]], dtype=float32),
 Affine(0.0005965576171875001, 0.0, -0.9283447265625001,
        0.0, -0.0003711274018448609, 51.89852871012711))

In [12]:
ndvi_output_path = data_dir + 'ndvi_reprojected.tif'

with rasterio.open(ndvi_output_path, 'w', **reprojected_profile) as dst:
    dst.write(ndvi_reprojected, 1)


In [13]:
import geopandas as gpd

gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs(dst_crs)


DataSourceError: /content/drive/MyDrive/london.shp: No such file or directory